<a href="https://colab.research.google.com/github/Varun-Mulchandani/roBERTa_based_SQuAD_QA/blob/master/roBERTa_SQuAD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Confirming GPU

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

For this project, I will be using HuggingFace's transformers library

In [2]:
!pip install transformers

     |████████████████████████████████| 645kB 2.6MB/s 
     |████████████████████████████████| 1.0MB 12.8MB/s 
     |████████████████████████████████| 3.8MB 17.1MB/s 
     |████████████████████████████████| 890kB 37.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=004c6765facc3daf77bf13ec04863aded1d5ab7645742935aaa4f914acaf8252
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Import necessary libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version', tf.__version__)

TF version 2.2.0-rc4


Max length for the input sequence is reduced to 384 due to lack of computaitonal resources for now.

In [0]:
MAX_LEN = 384

tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file = 'drive/My Drive/vocab-roberta-base.json',
    merges_file = 'drive/My Drive/merges-roberta-base.txt',
    lowercase = True,
    add_prefix_space = True
)

Loading the SQuAD dataset which has been converted to a csv file:

In [5]:
train = pd.read_csv('drive/My Drive/train (3).csv').fillna('')
train.head()

,Unnamed: 0,title,question,id,answers,answer_start,context
0,0,Beyoncé,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,1,Beyoncé,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,2,Beyoncé,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,3,Beyoncé,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,4,Beyoncé,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


In [0]:
rec = train.shape[0]  # Number of records in the training set
inputs = np.ones((rec, MAX_LEN), dtype = 'int32') # Input vector
attention_mask = np.zeros((rec, MAX_LEN), dtype = 'int32') # Attention Mask
token_type_ids = np.zeros((rec, MAX_LEN), dtype = 'int32') # Tokens produced
start_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # Start logit for answer
end_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # End logit for answer

for i in range(rec):

  context = ' '+' '.join(train.loc[i, 'context'].split())
  answer = ' '+' '.join(train.loc[i, 'answers'].split())
  question = ' '+' '.join(train.loc[i, 'question'].split())

  start_idx = train.loc[i, 'answer_start']

  chars = np.zeros((len(context)))
  chars[start_idx:start_idx + len(answer)] = 1
  if context[start_idx - 1] == ' ':
    chars[start_idx - 1] = 1
  
  enc1 = tokenizer.encode(context)
  enc2 = tokenizer.encode(question)

  # For resource limitations only.

  if len(enc1) + len(enc2) + 4 < MAX_LEN:

    #creating offsets
    offsets = []
    start_idx = 0

    for t in enc1.ids:
      w = tokenizer.decode([t])
      offsets.append((start_idx, start_idx + len(w)))
      start_idx += len(w)
    
    tokens = []
    for j, (a, b) in enumerate(offsets):
      sum_ = np.sum(chars[a:b])
      if sum_ > 0:
        tokens.append(j)

    # The input for roberta is in the form <s> Question </s></s> Answer </s>
    
    inputs[i, :len(enc1.ids) + len(enc2.ids) + 4] = [0] + enc2.ids + [2,2] + enc1.ids + [2]

    attention_mask[i, :len(enc1.ids) + len(enc2.ids) + 4] = 1

    if len(tokens) > 0:
      start_tokens[i, tokens[0] + 1] = 1
      end_tokens[i, tokens[-1] + 1] = 1

In [40]:
tokens

[0, 1, 2, 3, 4, 5]

In [0]:
def build_model():
  ids = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  att = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  tok = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)

  config = RobertaConfig.from_pretrained('drive/My Drive/config-roberta-base.json')
  bert_model = TFRobertaModel.from_pretrained('drive/My Drive/pretrained-roberta-base.h5', config = config)
  x = bert_model(ids, attention_mask=att, token_type_ids = tok)

  # For start logit

  x1 = tf.keras.layers.Dropout(0.1)(x[0])
  x1 = tf.keras.layers.Conv1D(1,1)(x1)
  x1 = tf.keras.layers.Flatten()(x1)
  x1 = tf.keras.layers.Activation('softmax')(x1)

  # For end logit

  x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
  x2 = tf.keras.layers.Conv1D(1,1)(x2)
  x2 = tf.keras.layers.Flatten()(x2)
  x2 = tf.keras.layers.Activation('softmax')(x2)

  # Initalising the model

  model = tf.keras.models.Model(inputs = [ids, att, tok], outputs = [x1, x2])
  optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-5)
  model.compile(loss='categorical_crossentropy', optimizer = optimizer)

  return model

In [0]:
model = build_model()

In [9]:
history = model.fit([inputs, attention_mask, token_type_ids], [start_tokens, end_tokens], epochs = 1, batch_size = 4, validation_split = 0.1)

19535/19535 [==============================] - 5016s 257ms/step - loss: 5.4522 - activation_7_loss: 2.7096 - activation_8_loss: 2.7427 - val_loss: 3.4484 - val_activation_7_loss: 1.7218 - val_activation_8_loss: 1.7265


In [12]:
print(enc1.ids)

[449, 2681, 14785, 257, 20887, 343, 36, 7203, 438, 238, 11, 645, 7, 3720, 758, 3115, 34, 2885, 41, 758, 3115, 3556, 18014, 36, 21163, 322, 6301, 438, 18, 78, 758, 1291, 21, 2885, 11, 14873, 19, 5, 343, 9, 364, 3252, 2552, 6, 1021, 46670, 6, 10409, 982, 4, 42, 1940, 34, 57, 617, 9094, 30, 10584, 4828, 4158, 19, 290, 97, 1947, 35, 475, 5992, 783, 7087, 343, 9, 1236, 20948, 6, 4533, 17419, 9, 5, 201, 102, 6, 1423, 1097, 261, 36, 30520, 18780, 3863, 43, 9, 127, 43918, 6, 3023, 118, 108, 260, 9, 5, 82, 18, 16441, 9, 1855, 1243, 6, 5251, 7771, 9, 12138, 42292, 6, 8, 19290, 46125, 9, 5, 7368, 16441, 9, 449, 33594, 4, 6301, 438, 18, 5891, 25541, 16, 7, 6292, 63, 10405, 19, 2241, 9636, 749, 6, 97, 758, 2244, 8, 171, 97, 538, 1947, 9, 5, 232, 7, 3042, 357, 4879, 1052, 8, 18477, 1767, 13, 449, 2681, 14785, 257, 4]


In [27]:
inp = 'Google LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, search engine, cloud computing, software, and hardware. It is considered one of the Big Four technology companies, alongside Amazon, Apple, and Facebook.'
que = 'What is Google?'
inp_id = np.zeros((1,MAX_LEN),dtype='int32')
attn_mask_input = np.zeros((1,MAX_LEN),dtype='int32')
token_type_id_input = np.zeros((1,MAX_LEN),dtype='int32')
inpenc = tokenizer.encode(inp)
queenc = tokenizer.encode(que)
print(len(que))
#chars_1 = np.zeros((len(inp)))
#chars_1[idx:idx + len(text2)] = 1

offset = []
id_ = 0
for t in inpenc.ids:
  w = tokenizer.decode([t])
  offset.append((id_, id_ + len(w)))
  id_ += len(w)
print(offset)

inp_id[0,:len(inpenc.ids)+len(queenc.ids) + 4] = [0] + queenc.ids + [2,2] + inpenc.ids + [2]
attn_mask_input[0,:len(inpenc.ids)+len(queenc.ids) + 4] = 1

15
[(0, 7), (7, 10), (10, 11), (11, 14), (14, 17), (17, 24), (24, 26), (26, 40), (40, 51), (51, 59), (59, 64), (64, 76), (76, 79), (79, 88), (88, 89), (89, 96), (96, 105), (105, 109), (109, 118), (118, 119), (119, 125), (125, 133), (133, 140), (140, 152), (152, 165), (165, 166), (166, 173), (173, 180), (180, 181), (181, 187), (187, 197), (197, 198), (198, 207), (207, 208), (208, 212), (212, 221), (221, 222), (222, 225), (225, 228), (228, 239), (239, 243), (243, 246), (246, 250), (250, 254), (254, 259), (259, 270), (270, 280), (280, 281), (281, 291), (291, 294), (294, 298), (298, 299), (299, 305), (305, 306), (306, 310), (310, 319), (319, 320)]


In [0]:
def generate_ans(inp,que,model,tokenizer):
  inp_id = np.zeros((1,MAX_LEN),dtype='int32')
  attn_mask_input = np.zeros((1,MAX_LEN),dtype='int32')
  token_type_id_input = np.zeros((1,MAX_LEN),dtype='int32')
  inpenc = tokenizer.encode(inp)
  queenc = tokenizer.encode(que)
  inp_id[0,:len(inpenc.ids)+len(queenc.ids) + 4] = [0] + queenc.ids + [2,2] + inpenc.ids + [2]
  attn_mask_input[0,:len(inpenc.ids)+len(queenc.ids) + 4] = 1
  s, f = model.predict([inp_id,attn_mask_input,token_type_id_input])
  s_ = np.argmax(s[0,])
  f_ = np.argmax(f[0,])
  ans = tokenizer.decode(inpenc.ids[s_ - 1: f_ + 1])

  return ans

In [64]:
inp = 'Google LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, search engine, cloud computing, software, and hardware. It is considered one of the Big Four technology companies, alongside Amazon, Apple, and Facebook.'
que = 'What is Google?'
ans = generate_ans(inp,que,model,tokenizer)
print(ans)

 google llc


Evidently, this model required more training and fine tuning (loss alone was pretty big.). In the next iteration, testing metric and more training.